<a href="https://colab.research.google.com/github/hxk271/SocDataSci/blob/main/W03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3 (반복문과 조건문)

Python에서 반복문과 조건문을 실제로 활용해보기 위해 우리나라 기후자료를 분석해보자. 다행히 우리 정부는 각종 공공 데이터를 제공하고 있으며, 가령 기온에 관한 자료는 우리나라 [기상자료개방포털](https://data.kma.go.kr)의 [기후통계분석]-[통계분석]-[기온분석]에서 누구나 무료로 다운받을 수 있다(회원 가입이 필요하다).
<br><br>
먼저 (1) 서울의 기온 자료를 사용하여 적절하게 <b>전처리(pre-processing)</b>하고, 다음으로 (2) 역대 최고기온이 어느날 몇 도였는지 찾아보기로 한다.

## 1. 파일 불러오기와 자료 전처리

> 이 수업에서는 구글 드라이브에 업로드되어 있는 파일을 사용하기로 한다. 그러기 위해서는 먼저 <code>gdown</code>이라는 라이브러리를 불러오고 파일을 다운로드해야 한다.
>
> 파일을 불러오기 위해서는 폴더(folder) 이름을 정확히 파악하고 지정할 줄 알아야 한다. 구글 서버에 다운로드된 파일이 어디에 있는지도 확인해보자. 좌측 패널에서 폴더 아이콘을 눌러보자.

In [ ]:
import gdown
link = 'https://drive.google.com/uc?id=12v_1pZ6zrGzWKcz-LmR-3hoQPKl7kA_P'
gdown.download(link)

> 이제부터 아래 창에 코드를 조금씩 채워 완성해 나가자!
>
> **CSV (comma-separated values)** 자료 파일을 읽어오려면 먼저 <code>csv</code> 라이브러리를 먼저 불러와야 한다. 그리고 그 안에서 <code>csv.reader</code>를 사용한다. 이때 <b>인코딩(encoding)</b>이라는 개념에 대해 조금 친숙해질 필요가 있다.

In [ ]:
import csv

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

> 반복문을 통해 데이터의 내용을 한 줄씩 살펴보자.

In [ ]:
for row in data:
    print(row)

> 한 번 파일을 열었으면 닫아야 한다. 그러지 않으면 해당 파일은 계속 *읽기 전용(read only)* 상태로 보호받게 된다(혹시 이로 인해 불편한 경험을 해본 적이 없을까?)

In [ ]:
file.close()

> 데이터의 변수(variable) 또는 컬럼(columns)을 <b>헤더(header)</b>라고도 부른다. 이걸 한 번 출력해보자.

In [ ]:
import csv

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#헤더
header = next(data)
print(header)

file.close()

> 아무리 생각해도 이건 헤더가 아니다. 좀 답답하니까, 이 자료를 여러분의 로컬 환경으로 다운로드받고 엑셀에서 열어보자. 하필 쓸떼없는 윗부분이 붙어있어서 원하는 결과가 제대로 나타나지 않았다(Why?).
>
> 첫 7줄을 스킵하고 다시 헤더를 살펴보자.

In [ ]:
import csv

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#첫 7줄은 쓸모없음
for i in range(0, 7):
    next(data)

#진짜 헤더
header = next(data)
print(header)

for row in data:
    print(row)

file.close()

> <b>문자열(string)</b>을 숫자로 바꾸어야만 수치 계산을 할 수 있다. 기온은 상식적으로 숫자니까 컴퓨터도 당연히 숫자로 인식했으리라고 어림짐작해선 안된다!
>
> 그런데 기온은 <b>정수(integer)</b>가 아니라 <b>실수(real number)</b>로 바꾸어야 한다(Why?). 실수를 컴퓨터에서 구현하기 위해 <b>부동소수점(floating points)</b>을 사용한다.

In [ ]:
import csv

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
print(header)

for row in data:

    #문자열인 최고기온을 실수로 변환
    row[-1] = float(row[-1])

    #결과 출력
    print(row)

file.close()

> 아래로 내려보면 결국 오류가 발생했다. 먼저 에러 메세지를 꼼꼼히 살펴보자. 빈 문자열 ''은 숫자가 될 수 없다(Why?).
>
> 엑셀에서 원인을 살펴보자. 특정 위치에서 자료의 공백이 있는데 이것이 변환되지 못해서 오류가 발생한 것 같다! 우리는 <b>결측치(missing value)</b>의 문제에 봉착했다. 본격적인 분석에 들어가기 앞서 빈칸을 그럴듯한 값으로 대체하는 과정을 <b>결측치 대체(missing value imputation)</b>라고 부른다(결측치 대체는 그 자체로 하나의 연구 분야이다).
>
> 여기서는 그냥 결측치에 -999 이라는 <b>플래그(flag)</b>를 세우고 넘어가기로 한다(이것은 흔한 관습이다). 다만 무슨 값이 들어가는지를 곰곰히 생각해보는 것이 좋다.

In [ ]:
import csv

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
print(header)

for row in data:

    #만약 결측치라면 -999를 넣을 것!
    if row[-1] =='':
        row[-1] = -999

    #문자열인 최고기온을 실수로 변환
    row[-1] = float(row[-1])

    #결과 출력
    print(row)

file.close()

> 또 오류가 발생했다! 마지막 관측치를 다시 확인해보자. 이번엔 <code>'\t'</code>이 이상하게 들어가 있다!
>
> <code>'\t'</code>는 <i>탭(tab)</i>을 뜻한다. 기상청 데이터 담당관의 과오로 모든 <i>관측치(observations)</i>는 탭으로 들여쓰기가 되어 있고, 이것이 '거의' 마지막 관측치 빈칸에도 반복되어 있다.
>
> 이것이 나타나면 어차피 자료가 끝났다는 뜻이다. 우리는 지난 주에 <code>break</code>를 써서 <i>반복문(loop)</i>에서 탈출할 수 있다는 것을 배웠다.

In [ ]:
import csv

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
print(header)

for row in data:

    #탭만 들어있는 관측치라면 중단
    if row[0] == '\t':
        break

    #만약 결측치라면 -999를 넣을 것!
    if row[-1] =='':
        row[-1] = -999

    #문자열인 최고기온을 실수로 변환
    row[-1] = float(row[-1])

    #결과 출력
    print(row)

file.close()

> 이제야 겨우 아무 문제도 없이 모든 관측치를 처음부터 끝까지 모두 살펴보았다!
>
> 나중에 가면 이런 귀찮은 일은 대체로 발생하지 않는다. <code>numpy</code>와 <code>pandas</code> 같은 데이터 분석 라이브러리 덕택에 자료관리는 사실 훨씬 쉽다. 그러니 너무 걱정하지 않아도 된다.

> **연습문제 1**. 이번엔 청주시 기온 공공 데이터를 사용하여 똑같은 코드를 작성하였다. 아래의 코드에 적절한 레이블을 붙이시오.
>
> 이 자료의 구글 드라이브 링크는 다음과 같다:
https://drive.google.com/uc?id=1hEwis-Wke7sQFbgmzeFjyWDMbkwN2Ar-

---
```python
import gdown
link = 'https://drive.google.com/uc?id=1hEwis-Wke7sQFbgmzeFjyWDMbkwN2Ar-'
gdown.download(link)
```

In [ ]:
import gdown
link = 'https://drive.google.com/uc?id=1hEwis-Wke7sQFbgmzeFjyWDMbkwN2Ar-'
gdown.download(link)

import csv
file = open('temp_chungju.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')
for i in range(0,7):
    next(data)
header = next(data)
print(header)
for row in data:
    if row[0] == '\t':
        break
    if row[-1] =='':
        row[-1] = -999
    row[-1] = float(row[-1])
    print(row)
file.close()

## 2. 문제 해결을 위한 알고리즘

> 한편 우리의 연구 질문은 ``서울의 최고기온은 언제 몇 도였나?'' 이므로 그 답을 찾아야 한다. 상식적으로 그 알고리즘이 어떠해야 할지 고민해 보자!
>
> 이 상황에서 굉장히 단순하지만 제법 유용한 알고리즘 하나가 있다! 그것은 **업데이트(update)** 방식이다. 즉 한줄한줄 데이터를 읽어내려가다가 조금이라도 예전보다 높은 기온이 나오면 이를 기록해둔다. 이 기록과 비교해가면서 다음 줄을 읽으면서 최고기온과 그 날짜를 업데이트해 나가는 것이다!

In [ ]:
import csv

max_temp = -999     #최고기온값이 기록될 placeholder
max_date = ''       #그 날이 언제인지 기록될 placeholder

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
#print(header)

for row in data:

    #탭만 들어있는 관측치라면 중단
    if row[0] == '\t':
        break

    #만약 결측치라면 -999를 넣을 것!
    if row[-1] =='':
        row[-1] = -999

    #문자열인 최고기온을 실수로 변환
    row[-1] = float(row[-1])

    #결과 출력(이젠 필요없음)
    #print(row)

    #만약 그날 최고기온이 저장해두었던 max_temp보다 높다면!
    if max_temp < row[-1]:
        max_date = row[0]        # 새롭게 갱신될 날짜
        max_temp = row[-1]       # 새롭게 갱신될 기온

file.close()

print('기상 관측 이래 서울의 최고기온이 가장 높았던 날은 ' + max_date + '이고 이날은 무려 ' + str(max_temp) + '도 였습니다.')

> 여기서는 <code>max_temp</code>가 -999로 처음에 엄청 낮게 설정되어 있다! 그리고 한줄 한줄 살펴보면서 그보다 조금이라도 높으면 <code>max_temp</code>와 <code>max_date</code>를 업데이트하고 있다.
>
> 한 가지 아쉬운 점은 "가장 높았던 날은"과 날짜 사이에 큰 빈칸이 있다는 점이다(Why?). 물론 쉽게 제거할 수 있다!

In [ ]:
import csv

max_temp = -999     #최고기온값이 기록될 placeholder
max_date = ''       #그 날이 언제인지 기록될 placeholder

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
#print(header)

for row in data:

    #탭만 들어있는 관측치라면 중단
    if row[0] == '\t':
        break

    #만약 결측치라면 -999를 넣을 것!
    if row[-1] =='':
        row[-1] = -999

    #문자열인 최고기온을 실수로 변환
    row[-1] = float(row[-1])

    #결과 출력(이젠 필요없음)
    #print(row)

    #만약 그날 최고기온이 저장해두었던 max_temp보다 높다면!
    if max_temp < row[-1]:
        max_date = row[0][1:]    # 새롭게 갱신될 날짜(\t만 슬쩍 건너뛰기)
        max_temp = row[-1]       # 새롭게 갱신될 기온

file.close()

print('기상 관측 이래 서울의 최고기온이 가장 높았던 날은 ' + max_date + '이고 이날은 무려 ' + str(max_temp) + '도 였습니다.')

> **연습문제 2-1**. 청주시 기온 공공 데이터를 사용하여 다음의 연구 질문에 답해보자
<br><br>
> ''청주의 최저기온이 가장 낮았을 때는 언제의 몇 도인가?''

In [ ]:
import csv

min_temp = 999      # 최저기온값이 기록될 placeholder
min_date = ''       # 그 날이 언제인지 기록될 placeholder

file = open('temp_chungju.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
#print(header)

for row in data:

    #탭만 들어있는 관측치
    if row[0] == '\t':
        break

    #만약 결측치라면 999를 넣을 것!
    if row[-2] =='':
        row[-2] = 999

    #문자열인 최저기온을 실수로 변환
    row[-2] = float(row[-2])

    #결과 출력(이젠 필요없음)
    #print(row)

    #만약 그날 최저기온이 저장해두었던 min_temp보다 낮다면!
    if min_temp > row[-2]:
        min_date = row[0][1:]
        min_temp = row[-2]

file.close()

print('기상 관측 이래 청주의 최저기온이 가장 낮았던 날은 ' + min_date + '이고 이날은 무려 ' + str(min_temp) + '도 였습니다.')

> **연습문제 2-2**. 청주시 기온 공공 데이터를 사용하여 다음의 연구 질문에 답해보자
<br><br>
> ''청주의 일교차가 가장 큰 날은 언제의 몇 도인가?''

In [ ]:
import csv

range_temp = 0      # 일교차가 기록될 placeholder
range_date = ''       # 그 날이 언제인지 기록될 placeholder

file = open('temp_chungju.csv', 'r', encoding = 'cp949')
data = csv.reader(file)

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
#print(header)

for row in data:

    #탭만 들어있는 관측치
    if row[0] == '\t':
        break

    #만약 결측치라면 999 또는 -999를 넣을 것!
    if row[-1] == '':
        row[-1] = -999   # -999를 넣어 빈 문자열이 있던 자리라고 표시
    if row[-2] == '':
        row[-2] = 999   # 999를 넣어 빈 문자열이 있던 자리라고 표시

    #문자열인 최고 또는 최저기온을 실수로 변환
    row[-1] = float(row[-1])
    row[-2] = float(row[-2])
    #print(row)

    #만약 그날 일교차가 저장해두었던 gap_temp보다 크다면!
    if range_temp < row[-1] - row[-2]:
        range_date = row[0][1:]
        range_temp = row[-1] - row[-2]

file.close()

print('기상 관측 이래 청주의 최고 일교차였던 날은 ' + range_date + '였고, 이날 일교차는 무려 ' + str(range_temp) + '도 였습니다.')

> **연습문제 2-3**. 서울시 기온 공공 데이터를 사용하여 다음의 연구 질문에 답해보자.
<br><br>
> ''서울에서 단 하루만에 평균기온이 가장 급격하게 오른 날은 언제이고 얼마나 올랐는가?''

In [ ]:
import csv

file = open('temp_seoul.csv', 'r', encoding = 'cp949')
data = csv.reader(file)

#몇가지 placeholders
record_diff = 0      # 하루만에 바뀐 평균온도차가 기록될 placeholder
record_date = ''     # 그 날이 언제인지 기록될 placeholder
yesterday = 13.5     # 첫 레코드(1907-10-01) 평균기온

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
#print(header)

for row in data:

    #탭만 들어있는 관측치
    if row[0] == '\t':
        break

    #만약 결측치라면 999 또는 -999를 넣기보다는 yesterday 값으로 대충 메움
    if row[2] == '':
        row[2] = yesterday

    #문자열인 평균기온을 실수로 변환
    row[2] = float(row[2])
    #print(row)

    #만약 그날 일교차가 저장해두었던 gap_temp보다 크다면!
    if record_diff < row[2] - yesterday:
        #print(row[2], yesterday)
        record_date = row[0][1:]           # \t는 건너뛰고 기록
        record_diff = row[2] - yesterday

    #이제 오늘은 어제로!
    yesterday = row[2]

file.close()

print('기상 관측 이래 서울에서 하루 만에 평균기온이 가장 크게 뛴 날은 ' + record_date + '였고, 그 전날과 비교하여 무려 ' + str(record_diff) + '도 올랐습니다.')

> **연습문제 2-4**. 청주시 기온 공공 데이터를 사용하여 다음의 연구 질문에 답해보자.
<br><br>
> ''역대 청주의 최고기온이 두번째로 높았던 날은 어느날 몇 도였는가?''

In [ ]:
import csv

max1_temp = 0      # 첫번째 최고기온값이 기록될 placeholder
max1_date = ''     # 그 날이 언제인지 기록될 placeholder
max2_temp = 0      # 두번째 최고기온값이 기록될 placeholder
max2_date = ''     # 그 날이 언제인지 기록될 placeholder

file = open('temp_chungju.csv', 'r', encoding = 'cp949')
data = csv.reader(file, delimiter = ',')

#첫 7줄은 쓸모없음
for i in range(0,7):
    next(data)

#진짜 헤더
header = next(data)
#print(header)

for row in data:

    #탭만 들어있는 관측치
    if row[0] == '\t':
        break

    #만약 결측치라면 -999를 넣을 것!
    if row[-1] =='':
        row[-1] = -999

    #문자열인 최고기온을 실수로 변환
    row[-1] = float(row[-1])

    #만약 그날 최고기온이 저장해두었던 max1_temp보다 낮다면!
    if row[-1] > max1_temp:

        #버리기 전에 max2_date와 max2_temp에 저장
        max2_date = max1_date
        max2_temp = max1_temp

        #평범한 업데이트
        max1_date = row[0][1:]
        max1_temp = row[-1]

file.close()

print('기상 관측 이래 청주에서 두번째로 최고기온이 높았던 날은 ' + max2_date + '이고 이날은 무려 ' + str(max2_temp) + '도 였습니다.')